# Daily Data Loading and FF5 Preparation

This notebook mirrors `data_loader.ipynb` but keeps everything at a **daily** frequency and joins to the new Fama-French 5-factor daily file. Output: `processed_data/daily_merged_with_ff5.parquet` containing daily IV skew, forward daily returns, and FF5 factors.


In [2]:
import pandas as pd
import numpy as np
import polars as pl
import os
from pathlib import Path

pd.set_option("display.max_columns", None)
print("✓ Libraries imported successfully")

✓ Libraries imported successfully


In [2]:
RAW_DATA_DIR = Path("raw_data")
PROCESSED_DATA_DIR = Path("processed_data")
PROCESSED_DATA_DIR.mkdir(exist_ok=True)

OPTIONS_PATH = RAW_DATA_DIR / "options_data.csv"
EQUITIES_PATH = RAW_DATA_DIR / "all_equities.csv"
MAPPING_PATH = RAW_DATA_DIR / "permno_secid_mapping.csv"
FF5_DAILY_PATH = RAW_DATA_DIR / "F-F_Research_Data_5_Factors_2x3_daily.csv"

DTE_LIMIT = 60
CALL_RANGE = (0.95, 1.05)
PUT_RANGE = (0.85, 1.05)

print("✓ Configuration set")
print(f"  Options path: {OPTIONS_PATH}")
print(f"  FF5 daily path: {FF5_DAILY_PATH}")


✓ Configuration set
  Options path: raw_data/options_data.csv
  FF5 daily path: raw_data/F-F_Research_Data_5_Factors_2x3_daily.csv


In [3]:
options_sample = pl.read_csv(OPTIONS_PATH, n_rows=10000)
print(f"Options data sample: {options_sample['date'].min()} to {options_sample['date'].max()}")
print(f"Columns: {options_sample.columns}")
options_sample.head()


Options data sample: 2019-01-02 to 2023-08-31
Columns: ['secid', 'date', 'exdate', 'cp_flag', 'strike_price', 'volume', 'open_interest', 'impl_volatility', 'opprc', 'moneyness', 'tte', 'close', 'spread', 'mod_open_interest', 'noi']


secid,date,exdate,cp_flag,strike_price,volume,open_interest,impl_volatility,opprc,moneyness,tte,close,spread,mod_open_interest,noi
i64,str,str,str,i64,i64,i64,f64,f64,f64,i64,f64,f64,i64,i64
5594,"""2021-10-07""","""2021-11-19""","""C""",10000,0,2,0.422947,1.25,0.907441,43,11.02,0.4,2,2
5594,"""2021-10-08""","""2021-11-19""","""C""",10000,0,2,0.456293,1.375,0.897666,42,11.14,0.15,2,0
5594,"""2021-10-11""","""2021-11-19""","""C""",10000,0,2,0.438753,1.275,0.904159,39,11.06,0.35,2,0
5594,"""2021-10-12""","""2021-11-19""","""C""",10000,0,2,0.453386,1.3,0.902527,38,11.08,0.4,2,0
5594,"""2021-10-13""","""2021-11-19""","""C""",10000,0,2,0.424783,1.225,0.906618,37,11.03,0.35,2,0


In [4]:
print("Loading equity returns...")
ret_df = (
    pl.read_csv(EQUITIES_PATH, schema_overrides={"RET": pl.Utf8})
    .with_columns(pl.col("RET").cast(pl.Float64, strict=False))
)
print(f"✓ Equity returns: {ret_df.shape}")
ret_df.head()


Loading equity returns...
✓ Equity returns: (10795504, 6)


PERMNO,date,TICKER,COMNAM,PRC,RET
i64,str,str,str,f64,f64
10026,"""2019-01-02""","""JJSF""","""J & J SNACK FOODS CORP""",141.0,-0.024829
10026,"""2019-01-03""","""JJSF""","""J & J SNACK FOODS CORP""",143.02,0.014326
10026,"""2019-01-04""","""JJSF""","""J & J SNACK FOODS CORP""",144.84,0.012725
10026,"""2019-01-07""","""JJSF""","""J & J SNACK FOODS CORP""",145.41,0.003935
10026,"""2019-01-08""","""JJSF""","""J & J SNACK FOODS CORP""",148.7,0.022626


In [5]:
map_df = pl.read_csv(MAPPING_PATH)
start_date = options_sample["date"].min()
filtered_map = map_df.filter(pl.col("edate") > start_date)
print(f"Mapping rows: {filtered_map.shape}")


Mapping rows: (16461, 4)


In [6]:
print("Loading FF5 daily factors...")
ff5_daily = pd.read_csv(FF5_DAILY_PATH)
ff5_daily[["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]] = ff5_daily[["Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"]] / 100
ff5_daily_pl = (
    pl.from_pandas(ff5_daily)
    .with_columns(
        pl.col("Date").cast(pl.Int64).cast(pl.Utf8).str.to_date("%Y%m%d").alias("date")
    )
    .select(["date", "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"])
)
print(f"✓ FF5 factors: {ff5_daily_pl.shape}")
ff5_daily_pl.head()


Loading FF5 daily factors...
✓ FF5 factors: (15667, 7)


date,Mkt-RF,SMB,HML,RMW,CMA,RF
date,f64,f64,f64,f64,f64,f64
1963-07-01,-0.0067,0.0,-0.0034,-0.0001,0.0016,0.0001
1963-07-02,0.0079,-0.0026,0.0026,-0.0007,-0.002,0.0001
1963-07-03,0.0063,-0.0017,-0.0009,0.0018,-0.0034,0.0001
1963-07-05,0.004,0.0008,-0.0027,0.0009,-0.0034,0.0001
1963-07-08,-0.0063,0.0004,-0.0018,-0.0029,0.0014,0.0001


In [7]:
def compute_daily_iv_skew(
    csv_path: str,
    dte_limit: int = 60,
    call_range: tuple[float, float] = (0.95, 1.05),
    put_range: tuple[float, float] = (0.85, 1.05),
    streaming: bool = True,
) -> pl.DataFrame:
    c = pl.col
    lf = pl.scan_csv(
        csv_path,
        schema_overrides={
            "secid": pl.Int64,
            "date": pl.Date,
            "tte": pl.Int32,
            "impl_volatility": pl.Float32,
            "cp_flag": pl.Categorical,
            "moneyness": pl.Float32,
        },
    ).select(["secid", "date", "tte", "impl_volatility", "cp_flag", "moneyness"])

    opt = (
        lf
        .filter(
            (c("tte") < dte_limit)
            & c("moneyness").is_not_null()
            & c("impl_volatility").is_finite()
        )
        .with_columns([
            pl.when(
                (c("cp_flag") == "C")
                & (c("moneyness") >= call_range[0])
                & (c("moneyness") <= call_range[1])
            ).then(c("impl_volatility")).otherwise(None).alias("call_iv"),
            pl.when(
                (c("cp_flag") == "P")
                & (c("moneyness") >= put_range[0])
                & (c("moneyness") <= put_range[1])
            ).then(c("impl_volatility")).otherwise(None).alias("put_iv"),
        ])
    )

    daily = (
        opt.group_by(["secid", "date"])
           .agg([
               pl.mean("call_iv").alias("call_iv_d"),
               pl.mean("put_iv").alias("put_iv_d"),
           ])
           .filter(c("call_iv_d").is_not_null() & c("put_iv_d").is_not_null())
           .with_columns((c("call_iv_d") - c("put_iv_d")).alias("IV_skew"))
           .select(["secid", "date", "IV_skew"])
    )

    return daily.collect(streaming=streaming)

print("Computing daily IV skew...")
daily_iv_df = compute_daily_iv_skew(
    str(OPTIONS_PATH),
    dte_limit=DTE_LIMIT,
    call_range=CALL_RANGE,
    put_range=PUT_RANGE,
)
print(f"✓ Daily IV skew rows: {daily_iv_df.shape}")
daily_iv_df.head()


Computing daily IV skew...
✓ Daily IV skew rows: (2074535, 3)


secid,date,IV_skew
i64,date,f32
206124,2020-05-27,0.013135
103138,2020-10-21,-0.016614
189573,2020-10-23,0.236785
136197,2021-04-16,-0.105985
137546,2023-07-14,-0.046991


In [8]:
print("Preparing daily returns with secid and forward shift...")
ret_with_ids = (
    ret_df.join(filtered_map, on="PERMNO")
    .with_columns(pl.col("date").str.to_date().alias("date"))
    .with_columns((pl.col("date") - pl.duration(days=1)).alias("prev_date"))
    .rename({"date": "next_date", "RET": "next_return"})
)
ret_with_ids = ret_with_ids.select([
    "PERMNO", "secid", "TICKER", "COMNAM", "PRC", "next_return", "next_date", "prev_date"
])
print(f"✓ Daily returns w/ mapping: {ret_with_ids.shape}")
ret_with_ids.head()


Preparing daily returns with secid and forward shift...
✓ Daily returns w/ mapping: (14832526, 8)


PERMNO,secid,TICKER,COMNAM,PRC,next_return,next_date,prev_date
i64,i64,str,str,f64,f64,date,date
10026,106500,"""JJSF""","""J & J SNACK FOODS CORP""",141.0,-0.024829,2019-01-02,2019-01-01
10026,106500,"""JJSF""","""J & J SNACK FOODS CORP""",143.02,0.014326,2019-01-03,2019-01-02
10026,106500,"""JJSF""","""J & J SNACK FOODS CORP""",144.84,0.012725,2019-01-04,2019-01-03
10026,106500,"""JJSF""","""J & J SNACK FOODS CORP""",145.41,0.003935,2019-01-07,2019-01-06
10026,106500,"""JJSF""","""J & J SNACK FOODS CORP""",148.7,0.022626,2019-01-08,2019-01-07


In [9]:
print("Joining IV skew (day t) with next-day returns...")
merged_daily = (
    daily_iv_df
    .rename({"date": "iv_date"})
    .join(ret_with_ids, left_on=["secid", "iv_date"], right_on=["secid", "prev_date"], how="left")
)
print(f"✓ After join: {merged_daily.shape}")
merged_daily.head()


Joining IV skew (day t) with next-day returns...
✓ After join: (2074541, 9)


secid,iv_date,IV_skew,PERMNO,TICKER,COMNAM,PRC,next_return,next_date
i64,date,f32,i64,str,str,f64,f64,date
206124,2020-05-27,0.013135,15145,"""BOX""","""BOX INC""",19.49,-0.007637,2020-05-28
103138,2020-10-21,-0.016614,86158,"""CTSH""","""COGNIZANT TECHNOLOGY SOLS CORP""",73.38,0.006722,2020-10-22
189573,2020-10-23,0.236785,null,null,null,null,null,null
136197,2021-04-16,-0.105985,null,null,null,null,null,null
137546,2023-07-14,-0.046991,null,null,null,null,null,null


In [10]:
print("Adding FF5 factors (matching next_date)...")
merged_with_ff5 = (
    merged_daily
    .join(ff5_daily_pl, left_on="next_date", right_on="date", how="left", suffix="_ff")
    .drop("date", strict=False)
    .with_columns((pl.col("next_return") - pl.col("RF")).alias("excess_return"))
)
print(f"✓ Final merged shape (pre-clean): {merged_with_ff5.shape}")
merged_with_ff5.head()


Adding FF5 factors (matching next_date)...
✓ Final merged shape (pre-clean): (2074541, 16)


secid,iv_date,IV_skew,PERMNO,TICKER,COMNAM,PRC,next_return,next_date,Mkt-RF,SMB,HML,RMW,CMA,RF,excess_return
i64,date,f32,i64,str,str,f64,f64,date,f64,f64,f64,f64,f64,f64,f64
206124,2020-05-27,0.013135,15145,"""BOX""","""BOX INC""",19.49,-0.007637,2020-05-28,-0.0041,-0.0209,-0.0241,-0.0025,0.0004,0.0,-0.007637
103138,2020-10-21,-0.016614,86158,"""CTSH""","""COGNIZANT TECHNOLOGY SOLS CORP""",73.38,0.006722,2020-10-22,0.0066,0.0095,0.0103,-0.0059,-0.0008,0.0,0.006722
189573,2020-10-23,0.236785,null,null,null,null,null,null,null,null,null,null,null,null,null
136197,2021-04-16,-0.105985,null,null,null,null,null,null,null,null,null,null,null,null,null
137546,2023-07-14,-0.046991,null,null,null,null,null,null,null,null,null,null,null,null,null


In [11]:
required_cols = [
    "IV_skew", "next_return", "excess_return",
    "Mkt-RF", "SMB", "HML", "RMW", "CMA", "RF"
]
complete_daily = merged_with_ff5.drop_nulls(required_cols)
print(f"Complete rows: {complete_daily.shape[0]:,} / {merged_with_ff5.shape[0]:,}")


Complete rows: 1,636,688 / 2,074,541


In [12]:
output_path = PROCESSED_DATA_DIR / "daily_merged_with_ff5.parquet"
complete_daily.write_parquet(output_path)
print(f"✓ Daily dataset saved to {output_path}")
print(f"  Rows: {complete_daily.shape[0]:,}")
print(f"  Unique secids: {complete_daily['secid'].n_unique()}")


✓ Daily dataset saved to processed_data/daily_merged_with_ff5.parquet
  Rows: 1,636,688
  Unique secids: 3949
